In [1]:
import spacy
from spacy import displacy
from IPython.display import display
import pandas as pd

nlp = spacy.load("fr_core_news_sm")

In [2]:
# Soit tout le texte d'un coup
fulltext = ""
with open("text1.txt", "r", encoding="utf-8") as text1:
    fulltext = text1.read()


doc = nlp(fulltext)

print("Nombre de token: ", len(doc))

Nombre de token:  1890


In [3]:
print("Graphe syntaxique de la première phrase du text1")
displacy.render(doc[:10], style="dep", jupyter=True)

Graphe syntaxique de la première phrase du text1


In [4]:
def print_token_info(token):
    print("Troisième token: ", token, f"({token.text}), index: ", token.i)
    print("Voisins: ", list(token.lefts), list(token.rights))

    print("Phrase: ", token.sent)
    print("Edges: ", token.left_edge, token.right_edge)
    print("Entity type: ", token.ent_type)
    print("Lemma: ", token.lemma, token.lemma_)
    print("Norm: ", token.norm, token.norm_)
    print("Shape: ", token.shape, token.shape_)
    print("Tag: ", token.tag, token.tag_)
    print("Dep: ", token.dep, token.dep_)
    print("Sentiment: ", token.sentiment)
    print("-----")


print_token_info(doc[5])
# print_token_info(doc[2])
# print_token_info(doc[7])
print_token_info(doc[0])

Troisième token:  couché (couché), index:  5
Voisins:  [Longtemps, ,, je, me, suis] [heure, .]
Phrase:  Longtemps, je me suis couché de bonne heure.
Edges:  Longtemps .
Entity type:  0
Lemma:  10940960452429598961 coucher
Norm:  11184698726792784299 couché
Shape:  13110060611322374290 xxxx
Tag:  100 VERB
Dep:  8206900633647566924 ROOT
Sentiment:  0.0
-----
Troisième token:  Longtemps (Longtemps), index:  0
Voisins:  [] []
Phrase:  Longtemps, je me suis couché de bonne heure.
Edges:  Longtemps Longtemps
Entity type:  0
Lemma:  10169700273540668978 longtemps
Norm:  10169700273540668978 longtemps
Shape:  16072095006890171862 Xxxxx
Tag:  86 ADV
Dep:  400 advmod
Sentiment:  0.0
-----


Insérer des données dans Neo4j

In [5]:
from neo4j import GraphDatabase, Session
from typing import Callable, Any

URI = "neo4j://localhost:7687"

Prise en main de Neo4j [Tutoriel Neo4j](https://neo4j.com/docs/python-manual/current/query-simple/)

In [6]:
driver = GraphDatabase.driver(URI)
tutorial_db = "tutorial"

In [7]:
insert = driver.execute_query("""
    CREATE (m:Monster {name: $monster})
    CREATE (b:Person {name: $name})
    CREATE (m)-[:ATE]->(b)
""", monster="Chopacabra", name="Cedric", database=tutorial_db).summary

print("Nodes created: ", insert.counters.nodes_created)

Nodes created:  2


In [ ]:
records, summary, keys = driver.execute_query(
    "MATCH (m:Monster)-[:ATE]->(p:Person) RETURN m.name as monster_name, p.name as person_eaten",
    database=tutorial_db,
)

for record in records:
    data = record.data()
    print(f'{data["monster_name"]} ate {data["person_eaten"]}')

 Monster Chopacabra ate Cedric


In [9]:
delete = driver.execute_query(
    "MATCH (a) DETACH DELETE a",
    database=tutorial_db,
)

print("Noeuds supprimés: ", delete.summary.counters.nodes_deleted, delete.records)

Noeuds supprimés:  1 []


In [10]:
driver.close()

Inserer les données du graphe de spacy

In [11]:
database = "neo4j"
driver = GraphDatabase.driver(URI)
session = driver.session(database=database)

In [60]:
# Cleanup
summary = driver.execute_query("Match (t: Token) DETACH DELETE t").summary
print(
    f"{summary.counters.nodes_deleted} noeuds et {summary.counters.relationships_deleted} liens supprimés"
)

417 noeuds et 466 liens supprimés


In [61]:
data = []

for token in doc:
    if token.dep_ == "ROOT":
        continue
    if token.is_punct or token.head.is_punct:
        continue
    if token.is_space or token.head.is_space:
        continue
    if token.is_stop or token.head.is_stop:
        continue
    if token.is_quote or token.head.is_quote:
        continue

    data.append(
        {
            "self": {"lemma": token.lemma_, "pos": token.pos_},
            "head": {
                "lemma": token.head.lemma_,
                "pos": token.head.pos_,
            },
            "relationship": token.dep_,
        }
    )

query = """
    UNWIND $tokens as token
    MERGE (self: Token { lemma: token.self.lemma, pos: token.self.pos })
    MERGE (head: Token { lemma: token.head.lemma, pos: token.head.pos })
    CREATE (self)-[:DEP { label: token.relationship }]->(head)
"""

result = session.execute_write(lambda tx: tx.run(query, tokens=data).to_eager_result())  # type: ignore

print(
    f"{result.summary.counters.nodes_created} noeuds et {result.summary.counters.relationships_created} liens crées"
)

412 noeuds et 455 liens crées


In [50]:
from spacy.lang.fr.stop_words import STOP_WORDS
print(STOP_WORDS)

{'sept', 'sera', 'était', 'on', 'via', 'hé', 'anterieure', 'facon', 'plus', 'différent', 'quelle', 'juste', 'dedans', 'son', 'ni', 'outre', 'antérieur', 'diverse', 'avant', 'sous', 'une', 'également', 'onze', 'lesquels', 'hors', 'desquelles', 'quatre', 'lorsque', 'soixante', 'vont', 'ouste', 'chacun', 'que', 'treize', 'apres', 'seulement', 'spécifiques', 'comment', 'auront', 'peux', 'certain', 'aux', 'hi', 'chez', 'qui', 'egalement', 'houp', 'cette', 'ceci', 'partant', "c'", 'durant', 'toi', 'ouias', 'possibles', 'six', 'dits', 'touchant', 'voilà', 'premièrement', 'a', 'avaient', 'là', 'quarante', 'autres', 'donc', 'plutôt', 'nombreuses', 'sur', 'longtemps', 'deuxième', 'mienne', 'dit', 'et', 'n’', 'étais', 'plutot', 'car', 'celle', "n'", 'pourrais', 'mien', 'pourrait', 'cet', 'dans', 'ô', 'lors', 'auxquels', 'dessus', 'autre', 'elles', 'celle-ci', 'hormis', 'ont', 'ceux-ci', 'vé', 'parlent', 'ouvert', 'quelles', 'envers', 'ces', 'douze', 'aurait', 'mes', 'quoi', 'première', 'aupres', 

Lister tous les verbes du corpus

In [14]:
query = "MATCH (t: Token {pos: 'VERB'}) RETURN t"
result = session.execute_read(lambda tx: tx.run(query).to_eager_result()) # type: ignore

print(f'{len(result.records)} verbes trouvés: ')
for result in result.records:
    verb = result["t"]
    print(f'{verb["lemma"]}')


119 verbes trouvés: 
chercher
éveiller
poser
croire
faire
lire
peser
devenir
étonner
trouver
doux
apparaître
éloigner
relever
décrire
étendre
hâter
suivre
oreiller
regarder
obliger
coucher
inconnu
apercevoir
lever
sonner
porter
donne
souffrir
entendre
porte
éteindre
falloir
rester
remède
rendormir
plonger
retourner
unir
dissiper
couper
naître
sentir
baiser
courbaturer
donner
retrouver
voir
désirer
imaginer
reculer
savoir
venir
asseoir
voyager
ouvrir
détendre
lâcher
tirer
agiter
réussir
tourner
remuer
repérer
induire
reconstruire
nommer
changer
hésiter
identifier
rappeler
deviner
allonger
finir
reposer
devoir
oublier
suspendre
figurer
représenter
revoir
fermer
vouloir
souffler
cesser
prendre
parler
survivre
choquer
commencer
détacher
appliquer
recouvrir
demander
graver
réveiller
réjouir
pouvoir
soulager
disparaître
partir
révolu
échapper
entourer
joue
arriver
connaître
évanouir
occuper
écouler
dormir
arrêter
estimer
suffire
frémir
dénuer
passer
présenter
endormir


Trouver les sujets d’un verbe donné : demander
Exemple : pour le verbe “manger”, afficher les tokens reliés par :DEP {label:"nsubj"}.

In [15]:
def subjects_of_verb(verb: str) -> list[str]:
    query = "MATCH (t: Token {pos: 'VERB', lemma: $verb})-[:DEP {label:'nsubj'}]-(s) RETURN s.lemma as subject"
    result = session.execute_read(lambda tx: tx.run(query, verb=verb).to_eager_result()) # type: ignore
    return list(map(lambda x: x["subject"], result.records))

print('Sujets pour le verbe "chercher":', subjects_of_verb("chercher"))
print('Sujets pour le verbe "devoir":', subjects_of_verb("devoir"))

Sujets pour le verbe "chercher": ['corps']
Sujets pour le verbe "devoir": ['esprit', 'malade']


Trouver les objets directs associés aux verbes
→ relations :DEP {label:"dobj"}.

In [16]:
query = "MATCH (t: Token {pos: 'VERB'})-[:DEP {label:'obj'}]-(o) RETURN t, o"
result = session.execute_read(lambda tx: tx.run(query).to_eager_result()) # type: ignore

print(f'{len(result.records)} verbes et objets trouvés: ')
for result in result.records:
    verb, object = result["t"], result["o"]
    print(f'{verb["lemma"]}, {object["lemma"]}')

42 verbes et objets trouvés: 
chercher, sommeil
poser, volume
faire, réflexion
trouver, obscurité
relever, distance
regarder, montre
donne, courage
entendre, sifflement
entendre, craquement
éteindre, gaz
rester, nuit
plonger, meuble
dissiper, jour,--date
naître, côte
naître, position
donner, entier
voir, cité
reculer, soleil
ouvrir, paupière
ouvrir, oeil
détendre, esprit
repérer, position
induire, direction
nommer, demeure
identifier, logis
rappeler, place
rappeler, genre
deviner, orientation
devoir, cheminée
oublier, flamme
oublier, fille
oublier, événement
souffler, lumière
prendre, tour
parler, rivalité
choquer, raison
recouvrir, vue
demander, heure
entourer, tête
écouler, jusqu’
frémir, animal
présenter, mur


Compter combien de fois chaque dépendance grammaticale apparaît


In [66]:
query = "MATCH (:Token)-[r]->() RETURN DISTINCT r.label as label, COUNT (r) as count"
result = session.execute_read(lambda tx: tx.run(query).to_eager_result()) # type: ignore
res = list(map(lambda x: (x["label"], x["count"]), result.records))

print(res)
print(len(res), sum(map(lambda x: x[1], res), 0))

[('amod', 37), ('obl:mod', 43), ('nmod', 72), ('nsubj', 27), ('acl:relcl', 20), ('dep', 6), ('obj', 53), ('advcl', 30), ('xcomp', 32), ('appos', 5), ('flat:name', 4), ('conj', 34), ('advmod', 32), ('obl:arg', 33), ('acl', 16), ('case', 4), ('aux:pass', 1), ('obl:agent', 5), ('nsubj:pass', 1), ('ccomp', 2), ('fixed', 1), ('mark', 2), ('parataxis', 1)]
23 461


Lister les adjectifs qui qualifient un nom donné : esprit, oiseau, train


In [67]:
def adjective_qualifying_name(noun: str) -> list[str]:
    query = """
        MATCH (adj:Token{pos: 'ADJ'})-[r]-(:Token{pos: 'NOUN', lemma: $noun}) 
        RETURN adj.lemma as lemma
    """
    result = session.execute_read(lambda tx: tx.run(query, noun=noun).to_eager_result())
    return list(map(lambda x: x["lemma"], result.records))

nouns = ["mur", "fauteuil", "chose"]
for noun in nouns:
    print(f'Adjectifs qui qualifient le nom "{noun}": ', adjective_qualifying_name(noun))

Adjectifs qui qualifient le nom "mur":  ['invisible']
Adjectifs qui qualifient le nom "fauteuil":  ['magique']
Adjectifs qui qualifient le nom "chose":  ['incompréhensible', 'obscur']


Mesurer les verbes les plus utilisés dans le corpus (avec une requête Cypher)


In [56]:
query = """
        MATCH (verb:Token{pos: 'VERB'})-[r]-()
        RETURN verb.lemma as verb, COUNT (r) as count 
        ORDER BY count DESC
        LIMIT 10
    """
result = session.execute_read(lambda tx: tx.run(query).to_eager_result())
res = list(map(lambda x: (x["verb"], x["count"]), result.records))

print(res)

[('devoir', 12), ('venir', 11), ('oublier', 9), ('trouver', 7), ('présenter', 7), ('chercher', 7), ('naître', 7), ('imaginer', 6), ('croire', 6), ('tirer', 6)]


Bonus:
Construire des couples Sujet–Verbe–Objet
Exemple : ("chat" - "mange" - "souris").

In [ ]:
def create_sentence(subject, verb, object: str):
    query = """
        MERGE (c:Token{lemma: $subject, pos: "NOUN"})
        MERGE (m:Token{lemma: $verb, pos: "VERB"})
        MERGE (s:Token{lemma: $object, pos: "NOUN"})
        MERGE (c)-[:DEP {label: "nsubj"}]->(m) 
        MERGE (s)-[:DEP {label: "obj"}]->(m) 
    """
    result = session.execute_write(
        lambda tx: tx.run(
            query, subject=subject, verb=verb, object=object
        ).to_eager_result()
    )
    print(
        result.summary.counters.nodes_created,
        "noeuds crées et",
        result.summary.counters.relationships_created,
        "liens crées",
    )

create_sentence("chat", "manger", "souris")
create_sentence("musicien", "jouer", "violon")
create_sentence("peintre", "peindre", "toile")

query = """
MATCH (subject: Token)-[:DEP {label: 'nsubj'}]->(verb:Token{pos: 'VERB'})<-[:DEP {label:'obj'}]-(object:Token)
RETURN subject.lemma as subject, verb.lemma as verb, object.lemma as object
"""
result = session.execute_read(lambda tx: tx.run(query).to_eager_result())
res = list(map(lambda x: (x["subject"], x["verb"], x["object"]), result.records))

print(res)

3 noeuds crées et 2 liens crées
3 noeuds crées et 2 liens crées
3 noeuds crées et 2 liens crées
[('corps', 'chercher', 'sommeil'), ('fauteuil', 'faire', 'réflexion'), ('Ève', 'naître', 'position'), ('femme', 'naître', 'position'), ('Ève', 'naître', 'côte'), ('femme', 'naître', 'côte'), ('sommeil', 'détendre', 'esprit'), ('lui,--mon', 'rappeler', 'place'), ('lui,--mon', 'rappeler', 'genre'), ('esprit', 'devoir', 'cheminée'), ('malade', 'devoir', 'cheminée'), ('sommeil', 'prendre', 'tour'), ('réflexion', 'prendre', 'tour'), ('église', 'parler', 'rivalité'), ('mémoire', 'présenter', 'mur'), ('chat', 'manger', 'souris'), ('musicien', 'jouer', 'violon'), ('peintre', 'peindre', 'toile')]


Bonus (dans l’ordre) : 
Calculez le graph de cooccurrence sous forme d’une matrice à partir du premier texte, importez ce graph dans neo4j puis mettez à jour ce graphe de cooccurrence depuis le deuxième texte (il y a des mots en commun entre ces deux paragraphes vous vous en doutez) et mettez à jour la base neo4j.
Faites des recherches sur Neo4j Graph Data Science (GDS Library) et trouvez un algorithme de recherche de chemin le plus court, faites-en la description et expliquer comment l’utiliser avec cette librairie officielle


In [ ]:
session.close()
driver.close()